In [4]:
from model_create import ModelGenerator, clear_folders
clear_folders()
# Параметры, которые необходимо расчитать:
parameters = ['FOPT', 'WOPR:*', 'FPR'] #, 'FWPT', 'FLPT', 'FGPT', 'FWIT']

# Зададим дату начала расчета и продолжительность:
start_date = "1 'JAN' 2020"
mounths = 12 # количество итераций;
days = 5 # продолжительность итерации;
# Зададим размеры модели:
lgr = True # локальное измельчение сетки по центру;
cells_cy = 20 # количество измельченных ячеек по y;
cells_cx = 4 # количество измельченных ячеек по x;
cells_v = 10 # размер измельченных ячеек, м;
nx = 30 # количество ячеек по х;
ny = 40 # количество ячеек по y;
nz = 10 # количество ячеек по z;
lx = 1500 # длина модели по x, м;
ly = lx # длина модели по y, м;
dz = 1 # размер ячейки по z, м;
tops_depth = 2500 # глубина залегания пласта, м;
# Зададим пористость и проницаемость:
por = 0.3 # пористость, д.ед;
permx = 200 # проницаемость по x, мД;
permy = permx # проницаемость по y, мД;
permz = permx*0.1 # проницаемость по z, мД;
# Зададим название, расположение добываюещей скважины и ее режим работы:
horizontal = [True] # Идентификатор горизонтальной скважины для скрипта
prod_names = ['P1'] # название скважин;
prod_xs=[15] # координата скважин по x;
prod_ys=[11] # координата скважин по y (если horizontal=true - начальная координата);
y_stop = [31] # координата конца горизонтальной скважины, если horizontal=true;
prod_z1s=[1] # начало интервала вскрытия по z;
prod_z2s=[6] # конец интервала вскрытия по z (если horizontal=true, то показывает уровень вскрытия);
rezim = ['BHP'] # Режим работы скважины
prod_q_oil = ['*'] # дебит, м3/сут;
prod_bhp = ['200'] # забойное давление, атм;
# Другие настройки модели:
only_prod = True # Модель только с добывающими скважинами
upr_rezim_water = False # Моделируем упруго-водонапорный режим
upr_rezim_gas = False # Моделируем газонапорный режим
# Задаем скин, радиус скважины (последовательно для добывающих и нагнетательных скважин):
skin = [0]
rw = [0.146]
# Свойства продукции:
oil_den = 860 # плотность нефти, кг/м3;
wat_den = 1010 # плотность воды, кг/м3;
gas_den = 0.9 # плотность газа, кг/м3;
density = [oil_den, wat_den, gas_den]
# EQUILIBRIUM DATA:
p_depth = 2500 # Глубина замера пластового давления, м;
p_init = 320  # Начальное пластовое давление, атм;
o_w_contact = 2550 # Глубина ВНК, м;
pc_woc = 0 # Капиллярное давление на ВНК 
g_o_contact = 2400 # Глубина ГНК, м;
pc_goc = 0 # Капиллярное давление на ГНК

model_name = f'TEST_MODEL_HORIZONTAL.0'
result_name = model_name
model = ModelGenerator(init_file_name='RIENM1_INIT.DATA',start_date=start_date, mounths=mounths,
                days=days, nx=nx, ny=ny, nz=nz, dz=dz, por=por, permx=permx,
                permy=permy, permz=permz, prod_names=prod_names, prod_xs=prod_xs,
                prod_ys=prod_ys, prod_z1s=prod_z1s, prod_z2s=prod_z2s, prod_q_oil=prod_q_oil,
                skin=skin, density=density, p_depth=p_depth, 
                p_init=p_init, o_w_contact=o_w_contact, pc_woc=pc_woc, g_o_contact=g_o_contact,
                pc_goc=pc_goc, tops_depth = tops_depth,
                rezim=rezim, prod_bhp=prod_bhp, horizontal=horizontal, y_stop=y_stop, only_prod=only_prod,
                lgr=lgr, lx=lx, ly=ly, cells_cy=cells_cy, cells_v=cells_v, cells_cx=cells_cx,
                upr_rezim_water=upr_rezim_water, upr_rezim_gas=upr_rezim_gas, rw=rw)
model.create_model(model_name, result_name, parameters)

DIMENS detected
DIMENS written
START detected
START written
DX detected
DX written
DY detected
DY written
DZ detected
DZ written
TOPS detected
TOPS written
PORO detected
PORO written
PERMX detected
PERMX written
PERMY detected
PERMY written
PERMZ detected
PERMZ written
DENSITY detected
DENSITY written
EQUIL detected
EQUIL written
WELSPECS detected
WELSPECS written
COMPDAT detected
COMPDAT written
WCONPROD detected
WCONPROD written
TSTEP detected
TSTEP written
TEST_MODEL_HORIZONTAL.0_RESULT.csv is created
TEST_MODEL_HORIZONTAL.0.csv is read
График построен и сохранен в атрибутах класса


In [5]:
model.summ_plot(['WOPR:P1'], name = [''])
model.iplot_fig()

Сравним полученные значения со стационарными формулами:

In [6]:
import math as m
pi = 3.14
Rк = lx/2 # м, радиус контура питания
dP = (p_init-int(prod_bhp[0]))/10*10**6 # Па, перепад давления
k = permx/10**15 # м2, проницаемость (в числовой модели изотропный пласт)
h = 10 # м, толщина пласта
L = 200 # м, длина горизонтального участка в пласте
mu = 0.0005 # Па*с, вязкость
rc = 0.073 # м, радиус скважины
Bo=1.2 # коэффициент объемного расширения
a = lx # м, сторона модели

# Вариант №1
# Батлер
Q4 = (2*3.14*k*h*dP)/(mu*(m.log1p(Rк/rc)-0.75))*86400/Bo
print('Приток по 1 варианту: ' + str(round(Q4)) + ' м3/сут')

# Вариант №2
# Батлер - приток на псевдоустановившемся режиме к протяженной горизонтальной скважины 147 с.
kh = k
kv = permz/10**15
Rm = 0.5*rc*(1+m.sqrt(kh/kv))
hm = h*m.sqrt(kh/kv)
Q5 = (6*kh*L*h*dP)/(mu*(a/2+3*hm/3.14*m.log1p(hm/2/3.14/Rm)))*86400/Bo
print('Приток по 2 варианту: ' + str(round(Q5)) + ' м3/сут')

# Вариант №3
# Батлер - неуст. режим в центре прямоуг зоны с учетов вертикального схождения
S=0
A = lx*ly
prow = A/(L**2) 
print('Условие выбора формулы: ' + str(prow))
if prow < 35:
    P = m.log1p(1+prow)/12
else:
    P = (m.log1p(prow)+0.1515)/(4*3.14)
P_ = hm/(2*3.14*L)*(m.log1p(hm/(2*3.14*Rm*m.sin(3.14/2)))+S)
Q1 = kh*h*dP/mu/(P_+P)*86400
print('Приток по 3 варианту: ' + str(round(Q1)) + ' м3/сут')

Приток по 1 варианту: 2557 м3/сут
Приток по 2 варианту: 4841 м3/сут
Условие выбора формулы: 56.25
Приток по 3 варианту: 9800 м3/сут
